In [ ]:
import numpy
import cv2
import os
import sys
from face_landmark_detection import faceLandmarkDetection
import csv
from math import cos, sin
import math

#For the frontal face detector
import dlib

#Antropometric constant values of the human head. 
#Found on wikipedia and on:
# "Head-and-Face Anthropometric Survey of U.S. Respirator Users"
#
#X-Y-Z with X pointing forward and Y on the left.
#The X-Y-Z coordinates used are like the standard
# coordinates of ROS (robotic operative system)
# P3D_RIGHT_SIDE = numpy.float32([-100.0, -77.5, -5.0]) #0
# P3D_GONION_RIGHT = numpy.float32([-110.0, -77.5, -85.0]) #4
# P3D_MENTON = numpy.float32([0.0, 0.0, -122.7]) #8
# P3D_GONION_LEFT = numpy.float32([-110.0, 77.5, -85.0]) #12
# P3D_LEFT_SIDE = numpy.float32([-100.0, 77.5, -5.0]) #16

P3D_FRONTAL_BREADTH_RIGHT = numpy.float32([-20.0, -56.1, 10.0]) #17
P3D_FRONTAL_BREADTH_LEFT = numpy.float32([-20.0, 56.1, 10.0]) #26
P3D_SELLION = numpy.float32([0.0, 0.0, 0.0]) #27



P3D_NOSE = numpy.float32([21.1, 0.0, -48.0]) #30
P3D_SUB_NOSE = numpy.float32([5.0, 0.0, -52.0]) #33
P3D_RIGHT_EYE = numpy.float32([-20.0, -65.5,-5.0]) #36
P3D_RIGHT_TEAR = numpy.float32([-10.0, -40.5,-5.0]) #39
P3D_LEFT_TEAR = numpy.float32([-10.0, 40.5,-5.0]) #42
P3D_LEFT_EYE = numpy.float32([-20.0, 65.5,-5.0]) #45
#P3D_LIP_RIGHT = numpy.float32([-20.0, 65.5,-5.0]) #48
#P3D_LIP_LEFT = numpy.float32([-20.0, 65.5,-5.0]) #54
P3D_STOMION = numpy.float32([10.0, 0.0, -75.0]) #62


#The points to track
#These points are the ones used by PnP
# to estimate the 3D pose of the face
# TRACKED_POINTS = (0, 4, 8, 12, 16, 17, 26, 27, 30, 33, 36, 39, 42, 45, 62)
TRACKED_POINTS = (17, 26, 27, 30, 33, 36, 39, 42, 45, 62)

ALL_POINTS = list(range(0,68)) #Used for debug only

cam_w = 1280
cam_h = 720

c_x = cam_w / 2
c_y = cam_h / 2
f_x = c_x / numpy.tan(60/2 * numpy.pi / 180)
f_y = f_x

#Estimated camera matrix values.
camera_matrix = numpy.float32([[f_x, 0.0, c_x],
                               [0.0, f_y, c_y], 
                               [0.0, 0.0, 1.0] ])

print("Estimated camera matrix: \n" + str(camera_matrix) + "\n")

#Distortion coefficients
camera_distortion = numpy.float32([0.0, 0.0, 0.0, 0.0, 0.0])


#This matrix contains the 3D points of the
# 11 landmarks we want to find. It has been
# obtained from antrophometric measurement
# on the human head.
# landmarks_3D = numpy.float32([P3D_RIGHT_SIDE,
#                               P3D_GONION_RIGHT,
#                               P3D_MENTON,
#                               P3D_GONION_LEFT,
#                               P3D_LEFT_SIDE,
#                               P3D_FRONTAL_BREADTH_RIGHT,
#                               P3D_FRONTAL_BREADTH_LEFT,
#                               P3D_SELLION,
#                               P3D_NOSE,
#                               P3D_SUB_NOSE,
#                               P3D_RIGHT_EYE,
#                               P3D_RIGHT_TEAR,
#                               P3D_LEFT_TEAR,
#                               P3D_LEFT_EYE,
#                               P3D_STOMION])

landmarks_3D = numpy.float32([
                              # P3D_GONION_RIGHT,
                              # P3D_MENTON,
                              # P3D_GONION_LEFT,
                              # P3D_LEFT_SIDE,
                              P3D_FRONTAL_BREADTH_RIGHT,
                              P3D_FRONTAL_BREADTH_LEFT,
                              P3D_SELLION,
                              P3D_NOSE,
                              P3D_SUB_NOSE,
                              P3D_RIGHT_EYE,
                              P3D_RIGHT_TEAR,
                              P3D_LEFT_TEAR,
                              P3D_LEFT_EYE,
                              P3D_STOMION])

#Declaring the two classifiers
my_face_detector = dlib.get_frontal_face_detector()

#  the trained shape predictor model
my_detector = faceLandmarkDetection('shape_predictor_68_face_landmarks.dat')

#Error counter definition
no_face_counter = 0

#Variables that identify the face
#position in the main frame.
face_x1 = 0
face_y1 = 0
face_x2 = 0
face_y2 = 0
face_w = 0
face_h = 0

#Variables that identify the ROI
#position in the main frame.
roi_x1 = 0
roi_y1 = 0
roi_x2 = cam_w
roi_y2 = cam_h
roi_w = cam_w
roi_h = cam_h
roi_resize_w = int(cam_w/10)
roi_resize_h = int(cam_h/10)




def findFace(frame):
  faces_array = my_face_detector(frame, 1)
  for f in faces_array:

    print(f)
  face = None
  lowestY = 0
  for i, pos in enumerate(faces_array):
    y1, y2 = pos.top(), pos.bottom()
    if (y1 + y2) / 2 > lowestY:
        lowestY = (y1 + y2) / 2
        face = pos
  return face

# def locateFace(frame):


def estimatePose(frame, frame_num):
  # face_x1 = pos.left()
  # face_y1 = pos.top()
  # face_x2 = pos.right()
  # face_y2 = pos.bottom()
# 641 344 901 662

  # with open('SGT026_childface.txt', newline='') as csvfile:
  # spamreader = csv.reader(landmarksFromFile, delimiter=' ')

  face = []
  landmarks_sgt = []
  yaw_pit_rol = []

  try:
    row = lookUpDict[frame_num]


  # for row in spamreader:
    # print(len(row))
    # if len(row) == 107:
    # if row[0] == '1040':
    #   print(row[0], frame_num, row[0] == frame_num, len(row) > 5)
    # print (row)
    if row[0] == frame_num and len(row) > 5:
      print('found')
      face = row[1:5]
      landmarks_sgt = row[5:-3]
      yaw_pit_rol = row[-3:]

      # face_x1 = 641
      # face_y1 = 344
      # face_x2 = 901
      # face_y2 = 662

      face_x1 = face[0]
      face_y1 = face[1]
      face_x2 = face[2]
      face_y2 = face[3]
  # landmarks_2D = my_detector.returnLandmarks(frame, face_x1, face_y1, face_x2, face_y2, points_to_return=TRACKED_POINTS)
  # print ('landmarks2d: ', landmarks_2D, type(landmarks_2D), landmarks_2D.shape)
      included = [0,9,10, 13,16,19,22,25,28,44]
      print(included)
      print(landmarks_sgt)
      print(yaw_pit_rol)
      points_sgt = [(int(float(landmarks_sgt[i])), int(float(landmarks_sgt[i+1]))) for i in range(0, len(landmarks_sgt), 2)]
      print(len(points_sgt))
      arr = numpy.zeros(20, dtype=numpy.float64).reshape(10,2,1)
      k = 0
      for i in included:
        arr[k][0] = points_sgt[i][0]
        arr[k][1] = points_sgt[i][1]
        k += 1


      landmarks_2D = arr
      print ('landmarks2d: ', landmarks_2D, type(landmarks_2D), landmarks_2D.shape)



      for point in landmarks_2D:
          cv2.circle(frame,( point[0], point[1] ), 2, (0,0,255), -1)


      #Applying the PnP solver to find the 3D pose
      # of the head from the 2D position of the
      # landmarks.
      #retval - bool
      #rvec - Output rotation vector that, together with tvec, brings 
      # points from the model coordinate system to the camera coordinate system.
      #tvec - Output translation vector.
      retval, rvec, tvec = cv2.solvePnP(landmarks_3D, 
                                        landmarks_2D, 
                                        camera_matrix, camera_distortion)
      # return rvec
      
      #Now we project the 3D points into the image plane
      #Creating a 3-axis to be used as reference in the image.
      axis = numpy.float32([[0,0,0], [0,50,0], [50,50,0], [50,0,0],
                       [0,0,50],[0,50,50],[50,50,50],[50,0,50] ])

      scl = 115
      axis = numpy.float32([[-scl,-scl,-scl], [-scl,scl,-scl], [scl,scl,-scl], [scl,-scl,-scl],
                       [-scl,-scl,scl],[-scl,scl,scl],[scl,scl,scl],[scl,-scl,scl] ]) 

      # axis = numpy.float32([[-2*scl,-scl,-scl], [-2*scl,scl,-scl], [0,scl,-scl], [0,-scl,-scl],
                       # [-2*scl,-scl,scl],[-2*scl,scl,scl],[0,scl,scl],[0,-scl,scl] ])

      imgpts, jac = cv2.projectPoints(axis, rvec, tvec, camera_matrix, camera_distortion)
      #Drawing the three axis on the image frame.
      #The opencv colors are defined as BGR colors such as: 
      # (a, b, c) >> Blue = a, Green = b and Red = c
      #Our axis/color convention is X=R, Y=G, Z=B
      sellion_xy = (landmarks_2D[7][0], landmarks_2D[7][1])

      # print ("sellion_xy: ", sellion_xy)
      # print ("imgpts: ", imgpts)

      imgpts = numpy.int32(imgpts).reshape(-1,2)
      # print(imgpts)

        # draw ground floor in green
      base = numpy.array([list(imgpts[0]), list(imgpts[4]), list(imgpts[5]), list(imgpts[1])]).reshape(-1, 2)
      top = numpy.array([list(imgpts[2]), list(imgpts[6]), list(imgpts[7]), list(imgpts[3])]).reshape(-1, 2)
      # for pts in imgpts:
      #   base.append(pts)
      # print base
      # print (base)
      # # print (imgpts[:4])
      frame = cv2.drawContours(frame, [base] ,-1,(0,0,255),3)
      # draw pillars in blue color
      for i,j in zip((0,1,4,5),(3,2,7,6)):
          frame = cv2.line(frame, tuple(imgpts[i]), tuple(imgpts[j]),(255),2)
      # draw top layer in red color
      frame = cv2.drawContours(frame, [top],-1,(0,255,0),2)

      # cv2.imshow("frame", frame)
      # k = cv2.waitKey(0)  & 0xff
      # if k == ord('s'):
      #   cv2.imwrite('3out.png', frame)
      #outFileName = imageFileName[:-4] + "headpose.png"
      #print outFileName
      #cv2.imwrite("/output/" + outFileName, frame)
      return frame
  except:
    return None

def draw_by_ypr_visage(lookUpDict, frame_no, frame):
  height, width, channels = frame.shape

  row = lookUpDict[frame_no]
  if len(row) > 8:
    face = row[1:4]
    pyr = row[4:7]
    
    pitch = float(pyr[0])
    yaw = float(pyr[1])
    roll = float(pyr[2])

    landmarks = row[7:]
    points_sgt = [(int(float(landmarks[i])*width), int((1-float(landmarks[i+1]))*height)) for i in range(0, len(landmarks)-1, 2)]
    cv2.circle(frame,( points_sgt[28][0], points_sgt[28][1] ), 2, (0,0,255), -1)


    # face_x = (1+float(face[0])) * width
    size = 300
    face_x = points_sgt[28][0] - size*0.5 
    face_y = points_sgt[28][1] - size*0.75
    
    p = -pitch / (180/math.pi)
    y = yaw / (180/math.pi)
    r = -roll / (180/math.pi)

    # x1 = 100 * (cos(y) * cos(r)) + width/2.0
    # y1 = 100 * (cos(p) * sin(r) + cos(r) * sin(p) * sin(y)) + height/2.0

    # x2 = 100 * (-cos(y) * sin(r)) + width/2.0
    # y2 = 100 * (cos(p) * cos(r) - sin(p) * sin(y) * sin(r)) + height/2.0

    # x3 = 100 * (sin(y)) + width/2.0
    # y3 = 100 * (-cos(y) * sin(p)) + height/2.0



    # print((x1, y1), (x2, y2), (x3, y3), width, height)

    # cv2.line(frame, (int(width/2), int(height/2)), (int(x1),int(y1)),(0,0,255),3)
    # cv2.line(frame, (int(width/2), int(height/2)), (int(x2),int(y2)),(0,255,0),3)
    # cv2.line(frame, (int(width/2), int(height/2)), (int(x3),int(y3)),(255,0,0),3)
    # cv2.line(frame, (int(x2), int(y2)), (int(x2+x3-width/2),int(y2+y3-height/2)),(255,0,0),3)
    # cv2.line(frame, (int(x1), int(y1)), (int(x1+x3-width/2),int(y1+y3-height/2)),(255,0,0),3)
    # cv2.line(frame, (int(x1), int(y1)), (int(x1+x2-width/2),int(y1+y2-height/2)),(255,0,0),3)
    # cv2.line(frame, (int(x3), int(y3)), (int(x3+x2-width/2),int(y3+y2-height/2)),(255,0,0),3)
    # cv2.line(frame, (int(x2), int(y2)), (int(x2+x1-width/2),int(y2+y1-height/2)),(255,0,0),3)
    # cv2.line(frame, (int(x3), int(y3)), (int(x3+x1-width/2),int(y3+y1-height/2)),(255,0,0),3)

    # cv2.line(frame, (int(x2+x1-width/2),int(y2+y1-height/2)), (int(x3+x1+x2-width),int(y3+y2+y1-height)),(255,0,0),3)
    # cv2.line(frame, (int(x3+x1-width/2),int(y3+y1-height/2)), (int(x3+x1+x2-width),int(y3+y2+y1-height)),(255,0,0),3)
    # cv2.line(frame, (int(x2+x3-width/2),int(y2+y3-height/2)), (int(x3+x1+x2-width),int(y3+y2+y1-height)),(255,0,0),3)
    

    x1 = size * (cos(y) * cos(r)) + face_x
    y1 = size * (cos(p) * sin(r) + cos(r) * sin(p) * sin(y)) + face_y

    x2 = size * (-cos(y) * sin(r)) + face_x
    y2 = size * (cos(p) * cos(r) - sin(p) * sin(y) * sin(r)) + face_y

    x3 = size * (sin(y)) + face_x
    y3 = size * (-cos(y) * sin(p)) + face_y



    print((x1, y1), (x2, y2), (x3, y3), face_x, face_y)
    # Draw base in red
    cv2.line(frame, (int(face_x), int(face_y)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(frame, (int(face_x), int(face_y)), (int(x2),int(y2)),(0,0,255),3)
    cv2.line(frame, (int(x2), int(y2)), (int(x2+x1-face_x),int(y2+y1-face_y)),(0,0,255),3)
    cv2.line(frame, (int(x1), int(y1)), (int(x1+x2-face_x),int(y1+y2-face_y)),(0,0,255),3)
    # Draw pillars in blue
    cv2.line(frame, (int(face_x), int(face_y)), (int(x3),int(y3)),(255,0,0),2)
    cv2.line(frame, (int(x1), int(y1)), (int(x1+x3-face_x),int(y1+y3-face_y)),(255,0,0),2)
    cv2.line(frame, (int(x2), int(y2)), (int(x2+x3-face_x),int(y2+y3-face_y)),(255,0,0),2)
    cv2.line(frame, (int(x2+x1-face_x),int(y2+y1-face_y)), (int(x3+x1+x2-2*face_x),int(y3+y2+y1-2*face_y)),(255,0,0),2)
    # Draw top in green 
    cv2.line(frame, (int(x3+x1-face_x),int(y3+y1-face_y)), (int(x3+x1+x2-2*face_x),int(y3+y2+y1-2*face_y)),(0,255,0),2)
    cv2.line(frame, (int(x2+x3-face_x),int(y2+y3-face_y)), (int(x3+x1+x2-2*face_x),int(y3+y2+y1-2*face_y)),(0,255,0),2)
    cv2.line(frame, (int(x3), int(y3)), (int(x3+x1-face_x),int(y3+y1-face_y)),(0,255,0),2)
    cv2.line(frame, (int(x3), int(y3)), (int(x3+x2-face_x),int(y3+y2-face_y)),(0,255,0),2)


    return frame

  return None

def draw_by_ypr_intra(lookUpDict, frame_no, frame):
  height, width, channels = frame.shape

  row = lookUpDict[frame_no]
  if len(row) > 5:
    face = row[1:5]
    # pyr = row[4:7]
    landmarks = row[5:-3]
    pyr = row[-3:]

    pitch = float(pyr[0])
    yaw = float(pyr[1])
    roll = 180-float(pyr[2])

    # landmarks = row[7:]
    points_sgt = [(int(float(landmarks[i])*width), int((1-float(landmarks[i+1]))*height)) for i in range(0, len(landmarks)-1, 2)]
    # cv2.circle(frame,( points_sgt[28][0], points_sgt[28][1] ), 2, (0,0,255), -1)


    # face_x = (1+float(face[0])) * width
    size = 100
    # face_x = points_sgt[28][0] - size*0.5 
    # face_y = points_sgt[28][1] - size*0.75

    face_x = width/2
    face_y = width/2
    
    p = -pitch / (180/math.pi)
    y = yaw / (180/math.pi)
    r = -roll / (180/math.pi)

    # x1 = 100 * (cos(y) * cos(r)) + width/2.0
    # y1 = 100 * (cos(p) * sin(r) + cos(r) * sin(p) * sin(y)) + height/2.0

    # x2 = 100 * (-cos(y) * sin(r)) + width/2.0
    # y2 = 100 * (cos(p) * cos(r) - sin(p) * sin(y) * sin(r)) + height/2.0

    # x3 = 100 * (sin(y)) + width/2.0
    # y3 = 100 * (-cos(y) * sin(p)) + height/2.0



    # print((x1, y1), (x2, y2), (x3, y3), width, height)

    # cv2.line(frame, (int(width/2), int(height/2)), (int(x1),int(y1)),(0,0,255),3)
    # cv2.line(frame, (int(width/2), int(height/2)), (int(x2),int(y2)),(0,255,0),3)
    # cv2.line(frame, (int(width/2), int(height/2)), (int(x3),int(y3)),(255,0,0),3)
    # cv2.line(frame, (int(x2), int(y2)), (int(x2+x3-width/2),int(y2+y3-height/2)),(255,0,0),3)
    # cv2.line(frame, (int(x1), int(y1)), (int(x1+x3-width/2),int(y1+y3-height/2)),(255,0,0),3)
    # cv2.line(frame, (int(x1), int(y1)), (int(x1+x2-width/2),int(y1+y2-height/2)),(255,0,0),3)
    # cv2.line(frame, (int(x3), int(y3)), (int(x3+x2-width/2),int(y3+y2-height/2)),(255,0,0),3)
    # cv2.line(frame, (int(x2), int(y2)), (int(x2+x1-width/2),int(y2+y1-height/2)),(255,0,0),3)
    # cv2.line(frame, (int(x3), int(y3)), (int(x3+x1-width/2),int(y3+y1-height/2)),(255,0,0),3)

    # cv2.line(frame, (int(x2+x1-width/2),int(y2+y1-height/2)), (int(x3+x1+x2-width),int(y3+y2+y1-height)),(255,0,0),3)
    # cv2.line(frame, (int(x3+x1-width/2),int(y3+y1-height/2)), (int(x3+x1+x2-width),int(y3+y2+y1-height)),(255,0,0),3)
    # cv2.line(frame, (int(x2+x3-width/2),int(y2+y3-height/2)), (int(x3+x1+x2-width),int(y3+y2+y1-height)),(255,0,0),3)
    

    x1 = size * (cos(y) * cos(r)) + face_x
    y1 = size * (cos(p) * sin(r) + cos(r) * sin(p) * sin(y)) + face_y

    x2 = size * (-cos(y) * sin(r)) + face_x
    y2 = size * (cos(p) * cos(r) - sin(p) * sin(y) * sin(r)) + face_y

    x3 = size * (sin(y)) + face_x
    y3 = size * (-cos(y) * sin(p)) + face_y



    print((x1, y1), (x2, y2), (x3, y3), face_x, face_y)
    # Draw base in red
    cv2.line(frame, (int(face_x), int(face_y)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(frame, (int(face_x), int(face_y)), (int(x2),int(y2)),(0,0,255),3)
    # cv2.line(frame, (int(x2), int(y2)), (int(x2+x1-face_x),int(y2+y1-face_y)),(0,0,255),3)
    # cv2.line(frame, (int(x1), int(y1)), (int(x1+x2-face_x),int(y1+y2-face_y)),(0,0,255),3)
    # Draw pillars in blue
    cv2.line(frame, (int(face_x), int(face_y)), (int(x3),int(y3)),(255,0,0),2)
    # cv2.line(frame, (int(x1), int(y1)), (int(x1+x3-face_x),int(y1+y3-face_y)),(255,0,0),2)
    # cv2.line(frame, (int(x2), int(y2)), (int(x2+x3-face_x),int(y2+y3-face_y)),(255,0,0),2)
    # cv2.line(frame, (int(x2+x1-face_x),int(y2+y1-face_y)), (int(x3+x1+x2-2*face_x),int(y3+y2+y1-2*face_y)),(255,0,0),2)
    # # Draw top in green 
    # cv2.line(frame, (int(x3+x1-face_x),int(y3+y1-face_y)), (int(x3+x1+x2-2*face_x),int(y3+y2+y1-2*face_y)),(0,255,0),2)
    # cv2.line(frame, (int(x2+x3-face_x),int(y2+y3-face_y)), (int(x3+x1+x2-2*face_x),int(y3+y2+y1-2*face_y)),(0,255,0),2)
    # cv2.line(frame, (int(x3), int(y3)), (int(x3+x1-face_x),int(y3+y1-face_y)),(0,255,0),2)
    # cv2.line(frame, (int(x3), int(y3)), (int(x3+x2-face_x),int(y3+y2-face_y)),(0,255,0),2)


    return frame

  return None
def draw_landmarks_visage(frame, landmarks):
  height, width, channels = frame.shape
  points_sgt = [(int(float(landmarks[i])*width), int((1-float(landmarks[i+1]))*height)) for i in range(0, len(landmarks)-1, 2)]
  # for i in range(0, len(landmarks_sgt), 2):
  # print(points_sgt)



  # omitted = [1,2,3,4,5,6,7,8,11,12,14,15,17,18, 20,21,23,24,26,27,29,30, 31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,48,49]
  # included = [0,9,10, 13,16,19,22,25,28,44]
  included = [9,0,10,13,16, 28, 25, 22, 19, 44] # For intraface
  included = [28] # For visage
  # included = [9,0,]

  omitted = [1,2,3,4,5,6,7,8,11,12,14,15,17,18, 20,21,23,24,26,27,29,30, 31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,48,49]
  # omitted = []
  idx = 0

  cv2.circle(frame,( points_sgt[28][0], points_sgt[28][1] ), 2, (0,0,255), -1)

  # for point in points_sgt:
  #   if idx in included:
  #     cv2.circle(frame,( point[0], point[1] ), 2, (0,0,255), -1)
  #   idx += 1
  cv2.imshow("frame", frame)
  k = cv2.waitKey(0)  & 0xff
  if k == ord('s'):
    cv2.imwrite('f249_annot.png', frame)
  return frame



# 249 frame
# pitch =  -0.385565
# yaw = -15.463036 
# roll = -1.559273

# 150 frame
# pitch = -0.414773 
# yaw = -16.106983 
# roll = -1.342264
def draw_one_pyr():
  file = open('SGT026_pivothead_visage.txt', newline='')
  csvreader = csv.reader(file, delimiter=' ')

  lookUpDict = {}
  for row in csvreader:
    lookUpDict[row[0]] = row
    if (len(row[7:]) % 2 == 0):
      print ("warning: ", len(row[7:]), row[0])

  pyr = lookUpDict['250'][4:7]
  pitch = float(pyr[0])
  yaw = float(pyr[1])
  roll = float(pyr[2])

  landmarks = lookUpDict['250'][7:]
  print(pyr)
  print(len(landmarks))

  frame = cv2.imread("f249.png")
  draw = draw_landmarks_visage(frame, landmarks)
  # draw = draw_by_ypr(yaw, pitch, roll, frame)
  cv2.imshow('frame', draw)
  k = cv2.waitKey(0)  & 0xff
  if k == ord('s'):
    cv2.imwrite('f249_annot.png', frame)


# draw_one_pyr()



# # Viz by landmarks
# points_sgt = [(int(float(landmarks[i])*width), int((1-float(landmarks[i+1]))*height)) for i in range(0, len(landmarks)-1, 2)]
# # for i in range(0, len(landmarks_sgt), 2):
# # print(points_sgt)



# # omitted = [1,2,3,4,5,6,7,8,11,12,14,15,17,18, 20,21,23,24,26,27,29,30, 31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,48,49]
# # included = [0,9,10, 13,16,19,22,25,28,44]
# included = [9,0,10,13,16, 28, 25, 22, 19, 44] # For intraface
# included = [9,0,10,13,16, 28, 25, 22, 19, 44] # For visage
# # included = [9,0,]
# arr = numpy.zeros(20, dtype=numpy.float64).reshape(10,2,1)
# print(arr, arr.shape)

# omitted = [1,2,3,4,5,6,7,8,11,12,14,15,17,18, 20,21,23,24,26,27,29,30, 31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,48,49]
# omitted = []
# idx = 0
# for point in points_sgt:
#   if idx not in omitted:
#     cv2.circle(frame,( point[0], point[1] ), 2, (0,0,255), -1)
#   idx += 1
# cv2.imshow("frame", frame)
# k = cv2.waitKey(0)  & 0xff
# if k == ord('s'):
#   cv2.imwrite('f249_annot.png', frame)
# -0.385565 -15.463036 -1.559273

# file = open('SGT026_pivothead_visage.txt', newline='')
file = open('SWC015_pivothead_visage.txt', newline='')
# with open('SGT026_childface.txt', newline='') as csvfile:
csvreader = csv.reader(file, delimiter=' ')

lookUpDict = {}
for row in csvreader:
  lookUpDict[row[0]] = row

# cap = cv2.VideoCapture('SGT026_2016_06_13_pivothead_merged.avi')
cap = cv2.VideoCapture('SWC015_2016_04_02_pivothead_AVI.avi')


# define the codec and create VideoWriter to write video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('SWC015_intra_ypr.avi',fourcc, 30.0, (1920,1080))

print(cap.isOpened())
counter = 1
while(cap.isOpened()):
  # print('here')
  ret, frame = cap.read()
  # face = findFace(frame)
  # print(face)
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  
  # print(counter)
  if counter < 10000:
    # draw = estimatePose(frame, str(counter))
    # draw = draw_by_ypr(lookUpDict, str(counter), frame)
    draw = draw_by_ypr_intra(lookUpDict, str(counter), frame)
    # cv2.imwrite('f249.png', frame)
    # break
    if draw != None:
      cv2.imshow('frame',draw)
      out.write(draw)
      if cv2.waitKey(25) & 0xFF == ord('q'):
          break
    else:
      cv2.imshow('frame',frame)
      out.write(frame)
      if cv2.waitKey(25) & 0xFF == ord('q'):
          break
  else:
    break
  counter += 1
  print('frame no.: ', counter + 1)


cap.release()
cv2.destroyAllWindows()

# arr = numpy.array([numpy.array([points_sgt[i][0], points_sgt[i][1]])  for i in included]).reshape(-1, 2)
# k = 0
# for i in included:
#   arr[k][0] = points_sgt[i][0]
#   arr[k][1] = points_sgt[i][1]
#   k += 1

# print ("arr is: ", arr, arr.shape)
# # arr = numpy.random.random((10,2,1))
# # print ("arr is: ", arr, arr.shape, arr.dtype)



# img_path = "f1410.png"
# frame = cv2.imread(img_path)
# # face = None
# # face = findFace(frame)
# # if face == None:
# #   # continue
# #   print("err")
# rvec = estimatePose(frame, '1410')

# cv2.imshow('frame',rvec)
# k = cv2.waitKey() & 0xFF



# print (arr)